In [1]:
# Import libs
import findspark
findspark.init("/home/george/spark-2.2.1-bin-hadoop2.7")

import os.path
import pandas
import math
import time

from metrics.Correlation import Correlation
from IO.Write import Write
from IO.Read import Read
from metrics.RSquare import RSquare
from DataSet.Dataset import DataSet

import pyspark
from pyspark.sql import SparkSession


import sys
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql.functions import countDistinct,avg,stddev
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                               OneHotEncoder , StringIndexer)

In [4]:
spark = SparkSession.builder.appName('melanoma').getOrCreate()

In [3]:
spark.stop()

In [5]:
path = '/home/george/diplwmatikh/code/maf/maf = 0.05/assocs/'
pathPatients = '/home/george/diplwmatikh/data/patients/Patients.txt'
pathSnp = '/home/george/diplwmatikh/code/snp/snp1.txt'

# Read ASSOCS

In [5]:

start_time = time.time()
#path = '/home/george/snp/'
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)
log_txt = sc.textFile(path)
sc.setLogLevel("WARN")
header = log_txt.first()
log_txt = log_txt.filter(lambda line: line != header)
temp_var = log_txt.map(lambda k: k.split())

inferRow = temp_var.map(lambda p: Row(CHR=(p[0].strip()),SNP=p[1].strip(),BP=p[2].strip(),A1=p[3].strip(),F_A=p[4].strip()
                                   ,F_U=p[5].strip(),A2=p[6].strip(),CHISQ=p[7].strip(),P=(p[8].strip())
                                      ,OR=p[9].strip()))
schemaString = "CHR SNP BP A1 F_A F_U A2 CHISQ P OR"

fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]
schema = StructType(fields)
schemaAssocs = spark.createDataFrame(inferRow,schema)
schemaAssocs.createOrReplaceTempView("Assocs")
#df=temp_var.toDF(header.split())
#log_df.select(log_df['field2']).show()
#log_df.filter(log_df['field2']=='1').count()
elapsed_time = time.time() - start_time
print(elapsed_time)
#df =spark.read.csv(path,header=True,inferSchema=True,escape="\t")

1.6540484428405762


# Read Patients

In [6]:
start_time = time.time()
#path = '/home/george/snp/'
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)
log_txt = sc.textFile(pathPatients)
sc.setLogLevel("WARN")
header = log_txt.first()
log_txt = log_txt.filter(lambda line: line != header)
temp_var = log_txt.map(lambda k: k.split())

inferRow = temp_var.map(lambda p: Row(eid=(p[0].strip()),sex=p[1].strip(),birth_year=p[2].strip(),cases=p[3].strip()))
schemaString = "eid sex birth_year cases"

fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]
schema = StructType(fields)
schemaPatients = spark.createDataFrame(inferRow,schema)
schemaPatients.createOrReplaceTempView("Patients")
#df=temp_var.toDF(header.split())
#log_df.select(log_df['field2']).show()
#log_df.filter(log_df['field2']=='1').count()
elapsed_time = time.time() - start_time
print(elapsed_time)
#df =spark.read.csv(path,header=True,inferSchema=True,escape="\t")

0.08335304260253906


# TO REMOVE NA

In [7]:
schemaAssocs = schemaAssocs.where("OR != 'NA'").drop()

In [8]:
schemaAssocs.createOrReplaceTempView("Assocs")


In [9]:
sqlDF = spark.sql("SELECT * FROM Assocs WHERE SNP=='.' ")
sqlDF.show()

+---+---+--------+-------+--------+---------+-------+-----+---------+-----+
|CHR|SNP|      BP|     A1|     F_A|      F_U|     A2|CHISQ|        P|   OR|
+---+---+--------+-------+--------+---------+-------+-----+---------+-----+
| 14|  .|20461026|      A|0.002976|0.0003825|      G|11.83|0.0005824|7.802|
| 14|  .|22587041|      G|0.002461|0.0002525|      A|11.22|0.0008103|9.766|
| 14|  .|25345143|      G|0.002465| 0.000127|      A|14.56|0.0001359|19.45|
| 14|  .|25463231|      A|0.002456|0.0002524|      T| 11.2|0.0008191|9.751|
| 14|  .|26400850|      C|0.001969|0.0001263|      T|10.92|0.0009498|15.61|
| 14|  .|27324500|      G|0.004429|0.0008847|  GGAGA|12.64|0.0003768|5.024|
| 14|  .|28245031|      A|0.003438|0.0005057|     AG| 12.6| 0.000385|6.819|
| 14|  .|28505286|      A|0.001965|0.0001264|      G| 10.9| 0.000964|15.58|
| 14|  .|28859881|ATTTTTT| 0.00247|0.0002532|      A|11.24|0.0008018| 9.78|
| 14|  .|30005281|   GTTC| 0.00197| 0.000127|      G|10.87|0.0009777|15.54|
| 14|  .|309

In [177]:
schemaPatients.show()

+-------+---+----------+-----+
|    eid|sex|birth_year|cases|
+-------+---+----------+-----+
|1000079|  0|      1957|    1|
|1000257|  1|      1940|    0|
|1000569|  1|      1945|    0|
|1004514|  0|      1939|    0|
|1005577|  0|      1961|    0|
|1007574|  0|      1965|    0|
|1008130|  1|      1944|    0|
|1008986|  1|      1947|    0|
|1009262|  0|      1947|    1|
|1009865|  1|      1953|    0|
|1009991|  0|      1953|    0|
|1012046|  1|      1952|    0|
|1013841|  0|      1959|    0|
|1015533|  1|      1941|    1|
|1016056|  1|      1959|    1|
|1017829|  1|      1941|    0|
|1019119|  0|      1949|    0|
|1019591|  1|      1939|    0|
|1021087|  0|      1961|    1|
|1023168|  1|      1955|    1|
+-------+---+----------+-----+
only showing top 20 rows



# STRING INDEXING

In [10]:
snp_indexer = StringIndexer(inputCol='SNP',outputCol='SNPIndex')
id_indexer = StringIndexer(inputCol='eid',outputCol='eidIndex')

In [11]:
Patients =id_indexer.fit(schemaPatients).transform(schemaPatients)

In [12]:
Patients.show()

+-------+---+----------+-----+--------+
|    eid|sex|birth_year|cases|eidIndex|
+-------+---+----------+-----+--------+
|1000079|  0|      1957|    1|  3762.0|
|1000257|  1|      1940|    0|  2124.0|
|1000569|  1|      1945|    0|  4454.0|
|1004514|  0|      1939|    0|  3467.0|
|1005577|  0|      1961|    0|  4290.0|
|1007574|  0|      1965|    0|  2813.0|
|1008130|  1|      1944|    0|  2277.0|
|1008986|  1|      1947|    0|  4877.0|
|1009262|  0|      1947|    1|   339.0|
|1009865|  1|      1953|    0|   782.0|
|1009991|  0|      1953|    0|  1263.0|
|1012046|  1|      1952|    0|  4436.0|
|1013841|  0|      1959|    0|  3043.0|
|1015533|  1|      1941|    1|  4084.0|
|1016056|  1|      1959|    1|   211.0|
|1017829|  1|      1941|    0|  3719.0|
|1019119|  0|      1949|    0|  1787.0|
|1019591|  1|      1939|    0|  4139.0|
|1021087|  0|      1961|    1|  1289.0|
|1023168|  1|      1955|    1|  3095.0|
+-------+---+----------+-----+--------+
only showing top 20 rows



In [13]:
Assocs = snp_indexer.fit(schemaAssocs).transform(schemaAssocs)

In [14]:
Assocs.show()

+---+----------+--------+-------+--------+---------+------+-----+---------+------+--------+
|CHR|       SNP|      BP|     A1|     F_A|      F_U|    A2|CHISQ|        P|    OR|SNPIndex|
+---+----------+--------+-------+--------+---------+------+-----+---------+------+--------+
| 14|         .|20461026|      A|0.002976|0.0003825|     G|11.83|0.0005824| 7.802|     0.0|
| 14| rs7154348|21245571|      G|  0.4102|   0.3659|     A|12.77|0.0003528| 1.205|  7696.0|
| 14|         .|22587041|      G|0.002461|0.0002525|     A|11.22|0.0008103| 9.766|     0.0|
| 14|         .|25345143|      G|0.002465| 0.000127|     A|14.56|0.0001359| 19.45|     0.0|
| 14|         .|25463231|      A|0.002456|0.0002524|     T| 11.2|0.0008191| 9.751|     0.0|
| 14|rs11159542|26355593|      A|  0.3929|    0.435|     C|11.37|0.0007451|0.8407|   577.0|
| 14| rs8008824|26355839|      C|   0.418|   0.4635|     T|13.56|0.0002313|0.8312|  1558.0|
| 14|rs33950250|26357534|      G|  0.4351|   0.4845|    GT|15.26| 9.38e-05|0.819

# GET IDS FROM PATINETS AND SNP

In [57]:
def setIdToName(aList):
    
    ids = {}
    nameToId = {}
    idToName = {}
    count = 0
    
    for i in aList:
        
        nameToId[i] = count
        idToName[count] = i
        count += 1
        
    ids['nameToId'] = nameToId
    ids['idToName'] = idToName
    
    yield ids

In [58]:
ids={}
test= Patients.select('eid')
#idss = test.toPandas().to_dict(orient='list')
snps = Assocs.select('SNPIndex')
idsa = snps.toPandas().to_dict(orient='list')
ids['patients'] = setIdToName(test.toPandas().to_dict(orient='list')['eid'])
ids['snps'] = idsa['SNPIndex']

In [38]:
X, Y = read.readSnpsCode(patients,ids)

NameError: name 'read' is not defined

In [53]:
test.toPandas().to_dict(orient='list')['eid']

['1000079',
 '1000257',
 '1000569',
 '1004514',
 '1005577',
 '1007574',
 '1008130',
 '1008986',
 '1009262',
 '1009865',
 '1009991',
 '1012046',
 '1013841',
 '1015533',
 '1016056',
 '1017829',
 '1019119',
 '1019591',
 '1021087',
 '1023168',
 '1024651',
 '1026239',
 '1026751',
 '1029754',
 '1035135',
 '1036509',
 '1037000',
 '1037914',
 '1038363',
 '1040071',
 '1040948',
 '1040997',
 '1045592',
 '1045787',
 '1047084',
 '1047823',
 '1048441',
 '1049125',
 '1050016',
 '1050928',
 '1050961',
 '1051955',
 '1052518',
 '1053271',
 '1054495',
 '1056314',
 '1057487',
 '1057583',
 '1058290',
 '1060623',
 '1063324',
 '1063894',
 '1068783',
 '1069215',
 '1069336',
 '1069629',
 '1070570',
 '1071394',
 '1071676',
 '1071998',
 '1072161',
 '1073565',
 '1074944',
 '1076510',
 '1077984',
 '1078122',
 '1078804',
 '1079551',
 '1080781',
 '1080957',
 '1082901',
 '1082955',
 '1087680',
 '1089775',
 '1092021',
 '1092497',
 '1092593',
 '1093953',
 '1095065',
 '1097797',
 '1098336',
 '1098478',
 '1099078',
 '11

# TESTS

In [6]:
data = spark.read.csv(pathSnp,inferSchema=True,header=True)

In [7]:
data.show(10)

+--------+-----------+-----------+----------+-----------+-----------+----------+----------+---------+---------+----------+-----------+----------+---------+----------+-----------+----------+---------+-----------+----------+-----------+---------+----------+-----------+---------+----------+---------+---------+----------+--------+---------+----------+----------+-----------+---------+--------+----------+-----------+----------+---------+----------+---------+--------+---------+-----------+----------+---------+-----------+----------+---------+-----------+-----------+----------+---------+---------+---------+----------------+-----------+---------+----------+----------+---------+---------+---------+-----------+---------+----------+---------+----------+-----------+-----------+----------+---------+----------+----------+-----------+----------+-----------+---------+---------+----------+---------+-----------+--------+-----------+----------+----------+----------+--------+----------+---------+----------

In [22]:
data.printSchema()

root
 |-- rs911957: integer (nullable = true)
 |-- rs572800017: integer (nullable = true)
 |-- rs587738238: integer (nullable = true)
 |-- rs11162706: integer (nullable = true)
 |-- rs550709613: integer (nullable = true)
 |-- rs189745502: integer (nullable = true)
 |-- rs12119972: integer (nullable = true)
 |-- rs12408458: integer (nullable = true)
 |-- rs6665807: integer (nullable = true)
 |-- rs3753431: integer (nullable = true)
 |-- rs12745266: integer (nullable = true)
 |-- rs545600098: integer (nullable = true)
 |-- rs12073252: integer (nullable = true)
 |-- rs6675652: integer (nullable = true)
 |-- rs76072566: integer (nullable = true)
 |-- rs587644581: integer (nullable = true)
 |-- rs61773304: integer (nullable = true)
 |-- rs6657631: integer (nullable = true)
 |-- rs746047175: integer (nullable = true)
 |-- rs74965151: integer (nullable = true)
 |-- rs114196385: integer (nullable = true)
 |-- rs1563827: integer (nullable = true)
 |-- rs76778551: integer (nullable = true)
 |-- 

In [9]:
data.createOrReplaceTempView("data")
1:4728382_CTG_C

In [16]:
#data = data.drop('1:4728382_CTG_C')
data = data.drop('1:226627216_CTT_C')
data = data.drop('1:152451771_TAGA_T')
data = data.drop('1:80107419_AG_A')
data = data.drop('2:76242103_AG_A')
data = data.drop('2:63620727_CA_C')
data = data.drop('2:63519267_CA_C')
data = data.drop('2:144454765_CAT_C')
data = data.drop('3:194391758_GTGAAT_G')
data = data.drop('3:53976329_GA_G')
data = data.drop('3:110678391_TC_T')
data = data.drop('3:194413082_AATTT_A')
data = data.drop('3:3959754_CA_C')
data = data.drop('3:60954392_TA_T')
data = data.drop('3:150226874_TTAAC_T')
data = data.drop('3:10816279_AC_A')

In [ ]:
22:47106694_AT_A

In [7]:
features = []

In [63]:
for i in data.columns:
    if 'rs' not in i:
        #f 'target' not in i:
        data = data.drop(i)
        print(i)
        break;

13:93790531_ATATTT_A


In [8]:
for i in data.columns:
    if 'rs'  in i:
        features.append(i)

In [9]:
p=0
for i in features:
    p+=1 
print(p)

12002


In [10]:
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                               OneHotEncoder , StringIndexer)

In [11]:
assempler = VectorAssembler(inputCols=features,outputCol='features')

In [12]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression


In [13]:
log_reg = LogisticRegression(featuresCol='features',labelCol='TARGET')


In [74]:
pipeline = Pipeline(stages=[assempler,log_reg])

train,test = data.randomSplit([0.7,0.3])


In [75]:
fit_model = pipeline.fit(train)

Py4JJavaError: An error occurred while calling o466.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 4.0 failed 1 times, most recent failure: Lost task 3.0 in stage 4.0 (TID 93, localhost, executor driver): java.lang.OutOfMemoryError: Java heap space
	at java.util.Arrays.copyOf(Arrays.java:3332)
	at java.lang.AbstractStringBuilder.ensureCapacityInternal(AbstractStringBuilder.java:124)
	at java.lang.AbstractStringBuilder.append(AbstractStringBuilder.java:448)
	at java.lang.StringBuilder.append(StringBuilder.java:136)
	at scala.StringContext.standardInterpolator(StringContext.scala:126)
	at scala.StringContext.s(StringContext.scala:95)
	at org.apache.spark.sql.catalyst.expressions.codegen.GenerateOrdering$.create(GenerateOrdering.scala:165)
	at org.apache.spark.sql.catalyst.expressions.codegen.GenerateOrdering$.create(GenerateOrdering.scala:43)
	at org.apache.spark.sql.catalyst.expressions.codegen.CodeGenerator.generate(CodeGenerator.scala:930)
	a

In [14]:
output = assempler.transform(data)

In [15]:
final_data = output.select('features','TARGET')

In [17]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [18]:
lr_model = log_reg.fit(train_data)

In [23]:
results = lr_model.transform(test_data)

In [19]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [20]:
evaluate = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='TARGET')

In [24]:
AUC = evaluate.evaluate(results)

In [25]:
AUC

1.0

In [26]:
train_data.count()

169322